In [ ]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
import umap
import matplotlib
import mygene
%matplotlib inline
import pickle
import sklearn
import random
import scanpy as sc
import torch
import torch.nn as nn
import torch.functional as Fx
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
from xgboost import XGBClassifier
# import sentence_transformers
plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Arial"
# })
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
np.random.seed(202310)
# use hnswlib for NN classification
try:
    import hnswlib
    hnswlib_imported = True
except ImportError:
    hnswlib_imported = False
    print("hnswlib not installed! We highly recommend installing it for fast similarity search.")
    print("To install it, run: pip install hnswlib")
from scipy.stats import mode

In [ ]:

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# set seed to control randomness
import pytorch_lightning as pl
pl.seed_everything(0)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

In [ ]:
adata_train = sc.read("/gpfs/gibbs/pi/zhao/wl545/pbmc/datasets/demo_train.h5ad")
adata_test = sc.read("/gpfs/gibbs/pi/zhao/wl545/pbmc/datasets/demo_test.h5ad")

In [ ]:

adata_comb = sc.concat([adata_train, adata_test], label = 'combinone', keys = ['train', 'test'])
adata_comb

In [ ]:


with open("/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/ensem_emb_gpt3.5all.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= list(adata_comb.var.index)
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings:
        lookup_embed[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1
lookup_embed.shape

# lookup_embed = np.random.rand(lookup_embed.shape[0], lookup_embed.shape[1])

adata_train =  adata_comb[adata_comb.obs.combinone == 'train']
adata_test =  adata_comb[adata_comb.obs.combinone == 'test']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder().fit(adata_train.obs.label)

train_obs, valid_obs = train_test_split(adata_train.obs_names, test_size=0.1, random_state=1 )
adata_train_train = adata_train[train_obs]
adata_train_valid = adata_train[valid_obs]

train_label = label_encoder.transform(adata_train_train.obs.label)
valid_label = label_encoder.transform(adata_train_valid.obs.label)


X_train = torch.FloatTensor(adata_train_train.X)

train_label = torch.FloatTensor(train_label)

batch_size = 512
lookup_embed = torch.FloatTensor(lookup_embed).to(device)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(lookup_embed.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, len(label_encoder.classes_))
        self.act = nn.ReLU()

    def forward(self, x, inputs):
        x = self.act(self.fc1(x))
        x = self.fc2(x) # can have dataset-specific gene embeddings
        emb = torch.matmul(inputs, x)
        label_out = self.fc3(emb)
        return label_out,emb



net = Net().to(device)
dataset = torch.utils.data.TensorDataset(X_train.to(device), train_label.to(device))
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)


import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)


In [ ]:
def model_evaluation(model, data, labels):
    model.eval()
    data = data.to(device)
    labels = labels.to(device)
    outputs,_ = net(lookup_embed, data)

    _, predicted = torch.max(outputs, 1)
    
    return (predicted == labels).sum().item() / len(labels)

def model_output(model, data):
    model.eval()
    data = data.to(device)
    outputs,emb = net(lookup_embed, data)

    _, predicted = torch.max(outputs, 1)
    
    return predicted.cpu().numpy(), emb.cpu().detach().numpy()

def eval_function(model, adata_train_train, adata_train_valid):
    model.eval()
    _,genePT_w_emebed_train =  model_output(model, torch.FloatTensor(adata_train_train.X))
    _,genePT_w_emebed_test = model_output(model, torch.FloatTensor(adata_train_valid.X))
    
    y_train = adata_train_train.obs.label
    y_test = adata_train_valid.obs.label
    
    # cell type clustering
    # very quick test
    k = 10  # number of neighbors
    ref_cell_embeddings = genePT_w_emebed_train
    test_emebd = genePT_w_emebed_test
    neighbors_list_gpt_v2 = []
    if hnswlib_imported:
        # Declaring index, using most of the default parameters from https://github.com/nmslib/hnswlib
        p = hnswlib.Index(space = 'cosine', dim = ref_cell_embeddings.shape[1]) # possible options are l2, cosine or ip
        p.init_index(max_elements = ref_cell_embeddings.shape[0], ef_construction = 200, M = 16)

        # Element insertion (can be called several times):
        p.add_items(ref_cell_embeddings, ids = np.arange(ref_cell_embeddings.shape[0]))

        # Controlling the recall by setting ef:
        p.set_ef(50) # ef should always be > k

        # Query dataset, k - number of closest elements (returns 2 numpy arrays)
        labels, distances = p.knn_query(test_emebd, k = k)

    idx_list=[i for i in range(test_emebd.shape[0])]
    gt_list = []
    pred_list = []
    for k in idx_list:
        # this is the true cell type
        gt = y_test[k]
        if hnswlib_imported:
            idx = labels[k]
        else:
            idx, sim = get_similar_vectors(test_emebd[k][np.newaxis, ...], ref_cell_embeddings)
        pred = mode(y_train[idx], axis=0)
        neighbors_list_gpt_v2.append(y_train[idx])
        gt_list.append(gt)
        pred_list.append(pred[0][0])
    acc = sklearn.metrics.accuracy_score(gt_list, pred_list)
    return acc

from pytorch_metric_learning import miners, losses
miner = miners.MultiSimilarityMiner()
loss_func = losses.TripletMarginLoss()

In [ ]:

prev = 0
net.train()
model_best = None
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs,emb = net(lookup_embed, inputs)
        
        loss = criterion(outputs, labels) + 100 * loss_func(emb, labels)
#         loss = loss_func(emb, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    
    if epoch % 5 ==0:
        eval_acc = eval_function(net, adata_train_train, adata_train_valid)
        print(eval_acc)
        if eval_acc > prev:
            prev = eval_acc
            model_best = pickle.loads(pickle.dumps(net))
        else:
            print("stop the training at:", epoch)
            break
print('Finished Training')

In [ ]:
input_data = torch.FloatTensor(adata_test.X)
labels = torch.FloatTensor()
label_predict,embeddings = model_output(model_best, input_data)
outlabel = label_encoder.inverse_transform(label_predict)

_,genePT_w_emebed_train =  model_output(model_best, torch.FloatTensor(adata_train.X))
_,genePT_w_emebed_test = model_output(model_best, torch.FloatTensor(adata_test.X))

y_train = adata_train.obs.label
y_test = adata_test.obs.label

In [ ]:

# cell type clustering
# very quick test
k = 10  # number of neighbors
ref_cell_embeddings = genePT_w_emebed_train
test_emebd = genePT_w_emebed_test
neighbors_list_gpt_v2 = []
if hnswlib_imported:
    # Declaring index, using most of the default parameters from https://github.com/nmslib/hnswlib
    p = hnswlib.Index(space = 'cosine', dim = ref_cell_embeddings.shape[1]) # possible options are l2, cosine or ip
    p.init_index(max_elements = ref_cell_embeddings.shape[0], ef_construction = 200, M = 16)

    # Element insertion (can be called several times):
    p.add_items(ref_cell_embeddings, ids = np.arange(ref_cell_embeddings.shape[0]))

    # Controlling the recall by setting ef:
    p.set_ef(50) # ef should always be > k

    # Query dataset, k - number of closest elements (returns 2 numpy arrays)
    labels, distances = p.knn_query(test_emebd, k = k)

idx_list=[i for i in range(test_emebd.shape[0])]
gt_list = []
pred_list = []
for k in idx_list:
    # this is the true cell type
    gt = y_test[k]
    if hnswlib_imported:
        idx = labels[k]
    else:
        idx, sim = get_similar_vectors(test_emebd[k][np.newaxis, ...], ref_cell_embeddings)
    pred = mode(y_train[idx], axis=0)
    neighbors_list_gpt_v2.append(y_train[idx])
    gt_list.append(gt)
    pred_list.append(pred[0][0])
sklearn.metrics.accuracy_score(gt_list, pred_list)

print('Precision, Recall, F1 (Marco weighted) for GenePT-w embedding: ', \
      sklearn.metrics.precision_recall_fscore_support(gt_list, pred_list,average='macro'))
